In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [2]:
# Cargamos el fichero desde hdfs
#accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/2018.csv',header=True)
accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)
#accidenteData = spark.read.csv ('file:///home/jose/Descargas/2018.csv',header=True)

In [18]:
import pyspark.sql.functions as func
from pyspark.sql.types import DoubleType

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData\
    .select(
        func.when (\
                (accidenteData['Nº']=='0'),\
#                pone calle 'esquina' calle
                func.regexp_replace (accidenteData['LUGAR ACCIDENTE'], '() - ', ' ESQUINA ')\
            )\
            .otherwise(
                func.concat (\
                    func.regexp_replace (accidenteData['LUGAR ACCIDENTE'],' NUM\s*$',' '), \
                    accidenteData['Nº'], \
                    func.lit(', MADRID, SPAIN')\
                )\
            )\
            .alias('direccion') ,\
#        func.when (\
#                (accidenteData['Nº']=='0'),\
#                None\
#            )\
#            .when (\
#                (accidenteData['LUGAR ACCIDENTE'].contains (' KM.')) & \
#                (accidenteData['Nº'].cast(DoubleType()) > 100) ,\
#                accidenteData['Nº'].cast(DoubleType()) / 1000\
#            )\
#            .otherwise(accidenteData['Nº'].cast(DoubleType()))\
#            .alias('numNorm') ,\
        accidenteData['LUGAR ACCIDENTE'].alias('calle'),\
        accidenteData['Nº'].alias('num')\
    )\
    .dropDuplicates ()



### Para geolocalizar las direcciones utilizamos la libreria geocoder de python.
### Se instala con el comando "pip install geocoder"

In [29]:
import geocoder
from pyspark.sql.types import StringType, MapType

g=geocoder.osm('calle de dolores barranco 44, madrid, spain')
def getGeo (address):
    geo = geocoder.osm (address)
    return geo.osm

func_getGeo = func.udf (getGeo,IntegerType())



In [30]:
g.osm
aux = accRed.select("direccion").limit(10).toPandas()
aux

,direccion
0,CALLE MAYOR ESQUINA PLAZA DEL COMANDANTE LAS ...
1,"CALLE DEL PRINCIPE CARLOS 23, MADRID, SPAIN"
2,CALLE DE LAS MARISMAS ESQUINA CALLE DE MARUJA ...
3,"CALLE DE VELAZQUEZ 17, MADRID, SPAIN"
4,"CALLE DE CARLINA 10, MADRID, SPAIN"
5,"CALLE DE CASTELLO 95, MADRID, SPAIN"
6,"CALLE DE VILLALONSO 47, MADRID, SPAIN"
7,"CALLE DE JORGE JUAN 59, MADRID, SPAIN"
8,"CALLE DE GERMAN PEREZ CARRASCO 27, MADRID, SPAIN"
9,"CALLE DE SEGOVIA 26, MADRID, SPAIN"


In [31]:
aux = accRed.limit(10)
accGeo = aux.withColumn('coordenadas',func_getGeo(accRed.direccion))


In [35]:
g=geocoder.osm('CALLE DEL PRINCIPE CARLOS 23, MADRID, SPAIN')
g.latlng
type (g.latlng[0])


float

In [33]:
accGeo.show()

+--------------------+--------------------+---+-----------+
|           direccion|               calle|num|coordenadas|
+--------------------+--------------------+---+-----------+
|CALLE  MAYOR ESQU...|CALLE  MAYOR - PL...|  0|       null|
|CALLE DEL PRINCIP...|CALLE DEL PRINCIP...| 23|       null|
|CALLE DE LAS MARI...|CALLE DE LAS MARI...|  0|       null|
|CALLE DE VELAZQUE...|CALLE DE VELAZQUE...| 17|       null|
|CALLE DE CARLINA ...|CALLE DE CARLINA ...| 10|       null|
|CALLE DE CASTELLO...|CALLE DE CASTELLO...| 95|       null|
|CALLE DE VILLALON...|CALLE DE VILLALON...| 47|       null|
|CALLE DE JORGE JU...|CALLE DE JORGE JU...| 59|       null|
|CALLE DE GERMAN P...|CALLE DE GERMAN P...| 27|       null|
|CALLE DE SEGOVIA ...|CALLE DE SEGOVIA ...| 26|       null|
+--------------------+--------------------+---+-----------+

